# Forecasting stream gap analysis for future surveys

Codebase & instructions for the model of Hendel & Bovy 2020, based primarily on the **streampepper** formalism of Bovy, Erkal & Sanders 2017

The method is designed to be extensible to new streams and/or survey parameters.

To add new streams, one needs
1. The stream's orbit, age, & velocity dispersion
2. Star counts in an existing survey
3. An assumed stellar population (age, metallicity, and stellar mass function slope)
4. A choice of coordinate system & mock survey area
5. (optional) To include background stars, a Galaxia (Sharma et al. 2011) mock halo survey at the stream position

To add a new survey, one needs
1. Isochrone tables (preferably matched to the Galaxia grid, see isochrone_handling.py)
2. An error model, e.g. $\Delta_{r}(r)$, in at least two photometric bands.

In [9]:
%matplotlib widget
import os
import sys
import time
import subprocess
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import galpy
import pickle
from astropy.table import Table,Column,Row
#set environment variables for your directory structure in .bash_profile, e.g.
#    export _FORECAST_DATA_DIR=/path/to/forecast/
_DATADIR  =  os.environ['_FORECAST_DATA_DIR']
_LOCALDIR =  os.environ['_FORECAST_LOCAL_DIR']

## Setting up stream models

First we create the streampepper dynamical models for each stream. This is done by make_streampepper_models file which has the necessary parameters and uses them to call the streammodel_util.py file. The latter generalizes the old pal5_util and gd1_util files. There will be a separate leading-tail and trailing-tail model for each. 

Depending on the potential, orbit, and desired number of times that impacts can happen these models can take quite some time to generate so the script can be run with multiprocessing enabled, if desired. It returns pickles of the models for quick restarts after the inital calculation.

New streams can be easily added by reproducing the config format seen for Pal5, Phoenix and GD-1.

Streammodel_util also allows for adding custom stream-oriented coordinate systems to the models as well as calculating the star count density in a selected 'observed' region. 

In [2]:
#Create models:
#run this in your command line (can take hours)
#python3 make_streampepper_models.py pal5 gd1 phx

In [2]:
from make_streampepper_models import pal5_config, gd1_config, phx_config

for config in [pal5_config, gd1_config, phx_config]:
    #load pickled models & coordinate systems into config.sdf
    config.load()
    
pal5_config.sdf['trailing']

Leading DF loaded
Trailing DF loaded
Leading DF loaded
Trailing DF loaded
Leading DF loaded
Trailing DF loaded


## Galaxia background model

Naturally [Galaxia](http://galaxia.sourceforge.net/Galaxia3pub.html) (Sharma et al. 2011) needs to be installed. To work with the resulting .ebf files one must use Python 2 for the ebfpy package. For me, pip2 install ebfpy was enough; we can call python2 with subprocess to generate a more convenient .fits format.

In [3]:
import bg
from astropy.io import fits as pyfits

for config in [pal5_config, gd1_config, phx_config]:
    #run Galaxia to calculate all stars in a 1 deg area around the progenitor 
    #position; creates a .fits table in outputDir for use with the isochrone handling
    config.bgfname =  _DATADIR+'galaxia_files/'+config.name+'_bg.fits'
    bg.run_galaxia(config.name+'_bg.ebf', config.name+'.param', 
                   outputDir = _DATADIR+'galaxia_files/', 
                   lon = config.obs.SkyCoord().galactic.l.value,
                   lat = config.obs.SkyCoord().galactic.b.value)                   

pal5_bg.ebf already exists!
gd1_bg.ebf already exists!
phx_bg.ebf already exists!


#### Load surveys and isochrones (set remake=True, save=True if you haven't created them before)

Use isochrone tables & error model to calculate how many of the stars in each background field will be confused with the stream's isochrone.


In [48]:
import survey_config
from survey_config import sdss_survey, cfht_survey, des_survey, lsst_survey, lsst10_survey, wfirst_survey

for config in [sdss_survey, cfht_survey, des_survey, lsst_survey, lsst10_survey, wfirst_survey]:
    config.load_iso_interps(remake=False, save=False, maxlabel=4)
    
streams = [pal5_config, gd1_config, phx_config]
surveys = [sdss_survey, cfht_survey, des_survey, lsst_survey, lsst10_survey, wfirst_survey]
d = np.zeros((len(streams), len(surveys)))

for i, stream in enumerate(streams):
    for j, survey in enumerate(surveys):
        d[i,j]=survey_config.calc_star_bg(stream, survey)
        #print(stream.name, survey.name, d[i,j])

t=Table()
t[' '] = ['SDSS', 'CFHT', 'DES', 'LSST 1-year', 'LSST 10-year', 'WFIRST']
t.add_columns(d.astype(int), names=['Pal 5', 'GD-1', 'Phoenix'])
t

,Pal 5,GD-1,Phoenix
str12,int64,int64,int64
SDSS,1403,269,531
CFHT,1472,131,407
DES,1384,49,370
LSST 1-year,681,76,216
LSST 10-year,531,84,186
WFIRST,273,25,93


## Stream sampling

## Approximate Bayesian Computation

In [6]:
print(_DATADIR+'model_pickles'+pal5_config.name+'_'+pal5_config.ntimes+'_leading.pkl')

/Users/hendel/projects/streamgaps/forecast/data/model_picklespal5_64sampling_leading.pkl
